 # <font color = #254117>[EEP 147]: ESG Analysis Notebook</font>

 <div style="width:image width px; font-size:80%; text-align:center;"><img src="big_creek.jpg" alt="alternate text" width="500" height="height" style="padding-bottom:0.5em;" />
<br>Big Creek Hydroelectric Project - Southern California Edison</div>

<br>
<font color = "red">Version 2025.2</font>

# Navigating this notebook

- This notebook can be utilized for analysis of the Electricity Strategy Game.
- Open the table of contents: View > Left Sidebar > Show Table of Contents.
- You can click Run > Run All Cells, then scroll to the bottom to use the interface to test adjustments to the demand and carbon prices.
- You can collapse all cells by clicking View > Collapse All Code. We suggest you start by looking at the notebook this way and expanding the sections as necessary.


# 1. Import packages we use
First on our agenda is to import **<font color = ##008700>dependencies</font>** -- packages in Python that add to the basic functions in Python.

In [71]:
from datascience import *
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, Dropdown, IntSlider, BoundedFloatText, HTML, GridspecLayout
import ipywidgets as widgets
from functools import partial
from IPython.display import display
from typing import List

plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = [10,6]
# Show all columns and rows of pandas DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
# warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# 2. Global Variables
Define variables that will be used throughout the notebook.

The variable `current_period` should contain the current round.

The variable `pab_periods` should contain each of the periods for which there was or will be a pay-as-bid auction.  This shouldn't change.

The variable `sections` is the list of section codes and should not be changed.

`price_floor` is the lowest allowed price in the carbon market. Set to `None` if there is no floor.

`price_ceiling` is the highest allowed price in the carbon market. Set to `None` if there is no floor.

`govt_allocations_456` is the list of total budgeted GHG emissions for each section in round 4, round 5, and round 6, respectively.

In [72]:
CURRENT_PERIOD = 2
SEASONS = [1,2,3,4] #! update to ['Winter', 'Spring', 'Summer', 'Fall']
N_HOURS_PER_SEASON = 4  #! Update to 24 hour seasonal profiles
PAB_PERIODS = [1]
SECTIONS = ['W9','T1','T2']

# Define carbon market parameters (only used in section 6, after Round 3 has finished)
price_floor = 24.04
price_ceiling = 88.24
# Gov budget for each round, for one section. Will be *number of sections
govt_allocations_456 = [23998+1391/3, 22570-1391/3, 18288]

# 3. Load demand, bids, and portfolio information
Next we load the files for Demand (realized and forecasted), Bids, Porfolios, and the Auction results into pandas dataframes.

**Demand:** 
- We load the CSV of projected demand, which also includes the realized demand for any rounds that have occured.
- For the current period and future periods, use the projected demand.
- For previous rounds, use the realized demand.

In [73]:
# Load the marginal cost bids and the ESG portfolios
BIDS_MC = pd.read_csv('Bids/MC_bids.csv').sort_values('PORTFOLIO')
ESG = pd.read_csv('ESGPorfolios.csv')

# Load the projected and realized demand for each hour of each round/period
DEMAND = pd.read_csv('demand.csv')

# Load the auction results if we've done the auction
if Path('auction_results.csv').exists():
    AUCTION_RESULTS = pd.read_csv('auction_results.csv')

## 3.1 Examine the portfolios of plants
Display the table of plants for a particular portfolio, which shows the costs, carbon intensity, and capacity of each plant.

`Var_Cost_USDperMWH` is the marginal cost of producing one additional MWh.

In [74]:
portfolios = ['Big Coal', 'Big Gas', 'Bay Views', 'Beachfront', 'East Bay', 'Old Timers', 'Fossil Light']
portfolio_to_examine = 'Old Timers'
display(ESG[ESG['Group'] == portfolio_to_examine])

,Group,Group_num,UNIT NAME,Capacity_MW,Fuel_Cost_USDperMWH,Var_OandM_USDperMWH,Var_Cost_USDperMWH,Carbon_tonsperMWH,FixedCst_OandM_perDay,Plant_ID
32,Old Timers,6,BIG CREEK,1000,0.00,0.0,0.00,0.00,15000,61
33,Old Timers,6,MOHAVE 1,750,30.00,4.5,34.50,0.94,15000,62
34,Old Timers,6,MOHAVE 2,750,30.00,4.5,34.50,0.94,15000,63
35,Old Timers,6,HIGHGROVE,150,49.11,0.5,49.61,0.58,0,64
36,Old Timers,6,SAN BERNADINO,100,53.44,0.5,53.94,0.63,0,65


## 3.2 Join Marginal costs, actual bids, and plant details
Next, we define a function to join the marginal-cost bids dataframe with any updated bids of completed rounds and the ESG plant details. We join based on the column `Plant_ID`. This function is used later in other helper functions.

In [75]:
# Define columns to use from the bid files (sometimes extra columns are present, so we need to specify which ones to use)
BIDS_COLS = ['TEAM', 'TEAM_ID', 'PORTFOLIO', 'PORTFOLIO_ID', 'PLANT', 'PLANT_ID', 'PERIOD', 'PRICE1', 'PRICE2', 'PRICE3', 'PRICE4']

def get_bids(section):
    """
    Retrieves bids data for a given section. 
    Merges the marginal cost bids with the actual bids for the given section's previous rounds.
    Then joins on the ESG data to get the plant information.

    Parameters:
    section (str): The section for which bids data is retrieved. Valid inputs are strings from the `sections` list.

    Returns:
    joined (dataframe): A dataframe containing the bids and plant information for the given section.
    """
    # Verify that section and current period are valid
    assert section in SECTIONS, "Invalid section"
    assert 0 <= CURRENT_PERIOD <= 7, "Invalid current period"

    # Copy the marginal cost bids, to update them with the actual bids for this period
    bids_all = BIDS_MC.copy().sort_values(['PLANT_ID', 'PERIOD'])

    # If current period is 0, there have not been any actual bids yet, so we can simply join on the ESG data and return
    if CURRENT_PERIOD == 0:
        return bids_all.merge(ESG, left_on="PLANT_ID", right_on='Plant_ID', how="left")

    # Load the bid file for the completed rounds. The last bid file has bids for all previous rounds
    try:
        bids_actual = (
            pd.read_csv(f'Bids/{section}_bids_{CURRENT_PERIOD-1}.csv', usecols=BIDS_COLS)
            .sort_values(['PLANT_ID', 'PERIOD'])
        )
    except FileNotFoundError:
        # If there is an error loading the files, return the bids_all dataframe
        s = f"""Missing bid CSVs for section {section} in the Bids folder.
            Current period is {CURRENT_PERIOD}. Has period {CURRENT_PERIOD-1} finished yet?
            Check if all bid files are present for periods 0 to {CURRENT_PERIOD-1} and that their names match the scheme used."""
        print(s)
        raise FileNotFoundError(s)

    # Update the hour1-hour4 price bids in the bids_all dataframe with the actual bids for any reounds before CURRENT_PERIOD
    for period_i in range(1, CURRENT_PERIOD):
        bids_all.loc[bids_all.PERIOD == period_i] = bids_actual.loc[bids_actual.PERIOD == period_i].values

    # Join on the ESG data to get the plant information
    joined = bids_all.merge(ESG, left_on="PLANT_ID", right_on='Plant_ID', how="left")
    return joined

# 4. Define helper functions

## 4.1 Fetch Demand
We will need to load demand for a given hour and period. Let's create a function that looks up the forecasted demand for a period and hour (or the realized demand if in the past). Demand is not certian, so let's also build in some functionality to allow the user to scale demand or replace demand with a different number.

In [76]:
# Show the demand table for the last completed round (4 hours)
display(DEMAND[DEMAND["round"] == CURRENT_PERIOD-1])
print(f'Demand for round 0, hour 1: {DEMAND[(DEMAND["round"] == 0) & (DEMAND["hour"] == 1)]["load"][0]} MWh')

,round,hour,load
4,1,1,10628.193726
5,1,2,17548.229423
6,1,3,18049.580398
7,1,4,16419.142708


Demand for round 0, hour 1: 12823.7013132679 MWh


In [77]:
def demand_calc(hour, period, demand_sp):
    """Return the energy demanded (load) for this period and hour, modified it by the demand_sp parameter. 

    Parameters:
    hour (int): The hour of the day (1-4).
    period (int): The period of the game (0-7).
    demand_sp (float): The demand adjustment, which can be a percentage change or a new demand value.
        If demand_sp is between -1 and 1, it is interpreted as a percentage change in demand. 
        If demand_sp is outside of this range, it is interpreted as the new demand value.

    Returns:
    demand (float): The modified demand for the given hour and period.
    """
    # Get the quantity demanded for this period and hour
    demand = DEMAND[(DEMAND["round"] == period) & (DEMAND["hour"] == hour)]["load"].values[0]
    # Adjust the demand by the demand_sp parameter, defined in Variable Definitions section below
    if -1 <= demand_sp <= 1:
        demand *= (1 + demand_sp)
    elif demand_sp > 1:
        demand = demand_sp
    else:
        raise ValueError("Invalid demand_sp value. Must be between -1 and 1 for percentage change, or >1 for new demand value.")
    return(demand)

## 4.2 Calculate price
Once we have the price bids for all plants and the demand in a period and hour, we can calculate the market clearing price. Below is a function to order the bids from low to high and find the market price where generation capacity equals the demand.

In [78]:
def price_calc(bids, demand, hour, period):
    """Calculate the price for a given level of demand, hour, and period, based on the price bids in the bids dataframe.

    Parameters:
    bids (dataframe): A dataframe containing the bids for each plant, for each period and hour.
    demand (float): The quantity of energy demanded.
    hour (int): The hour of the day (1-4).
    period (int): The period of the game (0-7).

    Returns:
    price (float): The price that clears the market for the given demand, hour, and period.
    """
    # Subset the input_df to only include the specified period, and sort the rows by the price
    sorted_bids = (
        # need to make a copy to avoid modifying the original dataframe
        bids.copy()
        # subset to only include the specified period
        .loc[bids["PERIOD"] == period]
        # sort the rows by the price bids for the specified hour, so smallest bid is first
        .sort_values(f"PRICE{hour}", ascending=True)
    )

    # Add a column to keep track of the cumulative capacity, which is the sum of the capacities of the plants up to that row
    sorted_bids['cumulative_capacity'] = sorted_bids['Capacity_MW'].cumsum()

    # Becuase this is sorted, we can simply take the cumulative sum of the Capacity_MW column
    # and find the first row where the cumulative capacity is greater than or equal to the demand
    # Then we return the price from that row
    price = sorted_bids.loc[sorted_bids['cumulative_capacity'] >= demand][f"PRICE{hour}"].values[0]

    return price

## 4.3 Plotting Helper Functions
We define some functions below that will help us plot the bid curves later.

In [79]:
def find_x_pos(widths):
    """Calculate the x positions on a plot for an array of widths.

    Parameters:
    widths (list): A list of widths of bars being plotted.

    Returns:
    list: A list of x positions where the bars will be positioned on the plot.
    """
    cumulative_widths = [0] + widths.cumsum().tolist()
    half_widths = [i/2 for i in widths]
    x_positions = []
    for i in range(len(widths)):
        x_positions.append(half_widths[i] + cumulative_widths[i])
    return x_positions


def price_line_plot(price):
    """Create a horizontal line on the current plot at the specified price."""
    plt.axhline(y=price, color='r', linewidth = 2)
    

def demand_plot(demand):
    """Create a vertical line on the current plot at the specified demand."""
    plt.axvline(x=demand, color='r', linewidth = 2)

## 4.4 Adjusting Marginal Costs and Bids
Define a function to apply a carbon tax and adjust the marginal cost of each plant based on it's carbon intensity.

Define a function to update the bids dataframe with alternative bids (used in a later section).

In [80]:
def adjust_by_cp(bids, hour, period, carbon_price):
    """
    Adjusts the bids dataframe by adding the carbon price to the variable cost and price columns.

    Parameters:
    bids (DataFrame): The input table to be adjusted.
    hour (int): The hour of the adjustment.
    period (int): The period of the adjustment.
    carbon_price (float): The carbon price to be applied to carbon intensity. Units are USD per ton of carbon.

    Returns:
    DataFrame: The adjusted table with the carbon price added to the variable cost and price columns.
    """
    bids_updated = bids.copy()
    # Add the marginal carbon tax burden to the variable cost
    bids_updated["Var_Cost_USDperMWH"] += carbon_price * bids_updated["Carbon_tonsperMWH"]
    # If adding to a future period, update the price bid (which is initially set at marginal cost for future periods)
    # If not a future period, leave the bids as submitted by students
    if (period >= CURRENT_PERIOD) | (CURRENT_PERIOD == 8):
        bids_updated["PRICE" + str(hour)] += carbon_price * bids_updated["Carbon_tonsperMWH"]
    return bids_updated
    

def user_defined_bids(bids, hour, period, my_portfolio, update_my_bids, update_others_bids, alternative_bids=None):
    """
    Updates the 'PRICE' column of the bids dataframe based on the specified bidding strategy.

    Args:
        bids (dataframe): The input table containing the data.
        hour (int): The hour for which the bids are being updated.
        period (int): The period for which the bids are being updated.
        my_portfolio (str): The name of the portfolio for which the bids are being updated.
        update_my_bids (bool or str): The bidding strategy for the 'my_portfolio' group. 
            If True, the bids are updated based on the values in the 'bids_myportfolio' dictionary.
            If True, a dataframe must be supplied in the 'alternative_bids' argument.
            If 'MC', the bids are updated based on the 'Var_Cost_USDperMWH' column of the bids dataframe.
        update_others_bids (bool or str): The bidding strategy for the other groups. 
            If True, the bids are updated based on the values in the respective 'bids_group' dictionaries.
            If 'MC', the bids are updated based on the 'Var_Cost_USDperMWH' column of the bids dataframe.
        alternative_bids (dataframe, optional): A dataframe containing alternative bids for the 'my_portfolio' group. See the example below for the required format.
            If update_my_bids is True, this argument must be supplied.

    Returns:
        dataframe: The updated dataframe with the 'PRICE' column modified based on the specified bidding strategy.
    """
    if (update_my_bids is False) & (update_others_bids is False):
        return bids
    # Make a local copy of the bids to be updated, and set the index to 'PLANT_ID' for matching updated bids
    bids_updated = bids.copy().set_index('UNIT NAME')
    # Filters
    period_filter = bids_updated["PERIOD"] == period
    myportfolio_filter = bids_updated["Group"] == my_portfolio
    not_myportfolio_filter = bids_updated["Group"] != my_portfolio
    
    # Update the price column based on the specified bidding strategy
    if update_my_bids is True:
        # Get the subset of the alternative bids dataframe for the 'my_portfolio' group, set the index to 'PLANT_ID' for updating the prices
        my_bids = alternative_bids[alternative_bids["Group"] == my_portfolio].copy().set_index('UNIT NAME')
        # Update this hour's price column for the 'my_portfolio' group
        bids_updated.update(my_bids[f'PRICE{hour}'])
    
    elif update_my_bids == 'MC':
        # Update this hour's price column for the 'my_portfolio' group based on the marginal costs
        bids_updated.loc[period_filter & myportfolio_filter, f"PRICE{hour}"] = bids_updated.loc[period_filter & myportfolio_filter, 'Var_Cost_USDperMWH']
    
    if update_others_bids is True:
        # Get the subset of the alternative bids dataframe for all non-'my_portfolio' groups, set the index to 'PLANT_ID' for updating the prices
        other_bids = alternative_bids[alternative_bids["Group"] != my_portfolio].copy().set_index('UNIT NAME')
        # Update this hour's price column for all non-'my_portfolio' groups
        bids_updated.update(other_bids[f'PRICE{hour}'])
                
    elif update_others_bids == 'MC':
        # Update this hour's price column for all non-'my_portfolio' groups based on the marginal costs
        bids_updated.loc[period_filter & not_myportfolio_filter, f"PRICE{hour}"] = bids_updated.loc[period_filter & not_myportfolio_filter, 'Var_Cost_USDperMWH']
    
    return bids_updated.reset_index()

## 4.5 Profits
Define a function to calculate the profits of a portfolio of plants based on the market clearing price. The first function calculates profits if we are in a uniform price market. The second function returns profits if we are in a pay-as-you-bid market.

**Uniform-Price Auction**<br>
If the market operates with a uniform-price auction, then all the plants with bids at or below the market clearing price are paid the market clearing price.

**Pay-As-Bid Auction**<br>
If the market operates with a PAB auction, then each plant with a bid at or below the market clearing price are *paid the price they bid*.

**Marginal Capacity**<br>
The total capacity available at exactly the market clearing price. 
For example, if the market clearing price is $40/MWh, the total capacity that was bid *at or under* $40 is 45,000 MW, 
and total capacity that was bid *under* $40 is 40,000 MW, then the marginal capacity (capacity offered at exactly $40/MWh)
is 5,000 MW.

**Marginal Demand**<br>
The amound of demand left over after all the non-marginal capacity has been supplied.
For example, if the market clearing price is $40, the total demand is 40,000 MW, and the sum of all capacity bid at prices less than $40 is 35,000 MW, 
then the marginal demand is 5,000 MW. There must be offered capacity at $40 that is 5,000 MW or greater.


In [81]:
def profit_calc(bids_period, hour, period, demand, price, my_portfolio):
    """Return the profit for a given portfolio, based on the demand and bids for a given period, hour.
    
    Parameters:
    bids_period (DataFrame): The dataframe containing the bids for the selected period.
    hour (int): The hour for which the profit is being calculated.
    period (int): The period for which the profit is being calculated. 
        The period is used to determine if the auction is a Pay-As-Bid or Uniform Price auction.
    demand (float): The quantity of energy demanded for the given hour and period.
    price (float): The price that clears the market for the given demand, hour, and period.
    my_portfolio (str): The name of the portfolio for which the profit is being calculated.

    Returns:
    profit (float): The profit for the given portfolio.
    """
    bids = bids_period.copy()
    
    # Calculate the proportion of the market's marginal capacity avaialable that satisfies marginal demand.
    nonmarginal_capacity = bids.loc[bids[f"PRICE{hour}"] < price, "Capacity_MW"].sum()
    marginal_capacity = bids.loc[bids[f"PRICE{hour}"] == price, "Capacity_MW"].sum()
    marginal_demand = demand - nonmarginal_capacity
    marginal_proportion = marginal_demand / marginal_capacity

    # Find my plants with bids below the market clearing price
    my_bids = bids[bids["Group"] == my_portfolio]
    nonmarginal = my_bids.loc[my_bids[f"PRICE{hour}"] < price]
    # Find my plants with bids at the market clearing price
    marginal = my_bids.loc[my_bids[f"PRICE{hour}"] == price]

    # Calculate the costs of operating my plants this round
    # Calculate the costs for the plants with bids below the market clearing price
    cost = (nonmarginal.Var_Cost_USDperMWH * nonmarginal.Capacity_MW).sum()
    # Add the costs for the plants with bids at the market clearing price, 
    # adjusted by the proportion of their total capacity that was used to satisfy demand
    cost += (marginal.Var_Cost_USDperMWH * marginal.Capacity_MW * marginal_proportion).sum()
    # Add the fixed costs for the round, divided by # of hours per round (all plants must pay their fixed costs regardless of operating)
    cost += my_bids.FixedCst_OandM_perDay.sum()/N_HOURS_PER_SEASON
    
    # Calculate the revenue for my portfolio, where the marginal capacity is adjusted by the proportion of the marginal demand
    if period in PAB_PERIODS:
        # Calculate the revenue in the Pay-As-Bid auction for an operating plant
        # Calculate the revenue for the plants with bids below the market clearing price
        revenue = (nonmarginal[f"PRICE{hour}"] * nonmarginal.Capacity_MW).sum()
        # Add the revenue for the plants with bids at the market clearing price,
        revenue += (marginal[f"PRICE{hour}"] * marginal.Capacity_MW * marginal_proportion).sum()
    else:
        # Calculate the revenue in the Uniform Price auction for an operating plant
        # Calculate the revenue for the plants with bids below the market clearing price
        revenue = (price * nonmarginal.Capacity_MW).sum()
        # Add the revenue for the plants with bids at the market clearing price,
        revenue += (price * marginal.Capacity_MW * marginal_proportion).sum()

    # Calculate the profit
    profit = revenue - cost
    return profit

In [82]:
def total_profits(section, my_portfolio, carbon_price_vec):
    # Merge auction results with input table
    portfolio_profit_dict = {}
    #auction_results_section = AUCTION_RESULTS.where("world_id", section)

    if CURRENT_PERIOD > 1:
        joined_table = get_bids(section)
        auction_results_section = AUCTION_RESULTS[AUCTION_RESULTS["world_id"] == section]
        # bids_all.merge(ESG, left_on="PLANT_ID", right_on='Plant_ID', how="left")
        full_table = joined_table.merge(auction_results_section, left_on="TEAM", right_on="team", how="left")
        portfolio_table = full_table.where("Group", my_portfolio)
        portfolio_profit = -portfolio_table.where("PERIOD", 1).to_df().loc[0, "adjustment"] * 1.05**(CURRENT_PERIOD-2)
        for period_i in range(1, CURRENT_PERIOD):        
            portfolio_table_period = joined_table.where("PERIOD", period_i).where("Group", my_portfolio)
            portfolio_profit_period = -sum(portfolio_table_period["FixedCst_OandM_perRound"])
            
            carbon_price = carbon_price_vec[period_i - 1]
            
            for hour_i in range(1, 5):
                demand = demand_calc(hour_i, period_i, 0)
                #TODO is this double applying the carbon price?
                joined_table = adjust_by_cp(joined_table, hour_i, period_i, carbon_price)
                sorted_joined_table = joined_table.where("PERIOD", period_i).sort("PRICE" + str(hour_i), descending = False)
                price = price_calc(sorted_joined_table, demand, hour_i, period_i)
                
                portfolio_profit_period += profit_calc(sorted_joined_table, hour_i, period_i, demand, price, my_portfolio)
                portfolio_profit_dict['Round ' + str(period_i)] = portfolio_profit_period
                    
            portfolio_profit += portfolio_profit_period * 1.05**(CURRENT_PERIOD - period_i - 1)
    else: 
        portfolio_profit = 0
    portfolio_profit_dict['Total'] = portfolio_profit
    
    output_df = pd.DataFrame.from_dict(portfolio_profit_dict, 
                                       orient = 'index', columns = [my_portfolio + ' Profit']).round().astype(int)
    
    return output_df

In [83]:
get_bids(SECTIONS[0])
AUCTION_RESULTS[AUCTION_RESULTS["game"] == SECTIONS[0]]

,game,team,portfolio,price
0,W9,Arrow,East Bay,300000
1,W9,Becker,Beachfront,280000
2,W9,Coase,Fossil Light,380000
3,W9,Debreu,Bay Views,240000
4,W9,Friedman,Big Gas,100000
5,W9,Heckman,Big Coal,100000
6,W9,Krugman,Old Timers,100000


In [84]:
def marketwide_profits_hour(bids, period, hour, price, demand):
    profits = 0
    bids = bids.sort_values(f'PRICE{hour}', ascending=True)
    
    for my_portfolio in portfolios:
        profits += profit_calc(bids, hour, period, demand, price, my_portfolio)
    
    return profits

## 4.6 Emissions
Calculate total emissions for all *operating* plants in a group.

In [85]:
def emissions_calc(bids_period, hour, demand, price, my_portfolio):    
    bids = bids_period.copy()
    
    # Calculate the marginal capacity and demand
    nonmarg_capacity = bids.loc[bids[f"PRICE{hour}"] < price, "Capacity_MW"].sum()
    marg_capacity = bids.loc[bids[f"PRICE{hour}"] == price, "Capacity_MW"].sum()
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    # Calculate the emissions, where the marginal capacity is adjusted by the proportion of the marginal demand
    my_bids = bids[bids["Group"] == my_portfolio]
    emissions = 0
    for _, row in my_bids.loc[my_bids[f"PRICE{hour}"] < price].iterrows():
        emissions += row["Carbon_tonsperMWH"] * row["Capacity_MW"]
    for _, row in my_bids.loc[my_bids[f"PRICE{hour}"] == price].iterrows():
        emissions += row["Carbon_tonsperMWH"] * row["Capacity_MW"] * marg_proportion
    return emissions

In [86]:
def total_emissions(section, my_portfolio):
    portfolio_emissions_dict = {}
    if CURRENT_PERIOD > 1:
        joined_table = get_bids(section)
        portfolio_emissions = 0
        for period_i in range(1, CURRENT_PERIOD):        
            portfolio_emissions_period = 0
            
            for hour_i in range(1, 5):
                demand = demand_calc(hour_i, period_i, 0)
                sorted_joined_table = joined_table.where("PERIOD", period_i).sort("PRICE" + str(hour_i), descending = False)
                price = price_calc(sorted_joined_table, demand, hour_i, period_i)
                
                portfolio_emissions_period += emissions_calc(sorted_joined_table, hour_i, demand, price, my_portfolio)
                portfolio_emissions_dict['Round ' + str(period_i)] = portfolio_emissions_period
                    
            portfolio_emissions += portfolio_emissions_period
    else: 
        portfolio_emissions = 0
    portfolio_emissions_dict['Total'] = portfolio_emissions
    
    output_df = pd.DataFrame.from_dict(portfolio_emissions_dict, 
                                       orient = 'index', columns = [my_portfolio + ' Emissions']).round().astype(int)
    
    return output_df

In [87]:
def marketwide_emissions_hour(bids, hour, price, demand):
    emissions = 0
    bids = bids.sort_values(f'PRICE{hour}', ascending=True)
    for my_portfolio in portfolios:
        emissions += emissions_calc(bids, hour, demand, price, my_portfolio)
                        
    return emissions

# 5 Plotting the Market and Portfolio

## 5.1 Plotting Functions

In [88]:
ENERGY_COLORS_DICT = {'Bay Views' : '#EC5F67', 'Beachfront' : '#F29056', 'Big Coal' : '#F9C863', 'Big Gas' : '#99C794', 
                      'East Bay' : '#5FB3B3', 'Fossil Light' : '#6699CC', 'Old Timers' : '#C594C5'}

In [89]:
def market_plot(bids_period, hour, period, demand, price):
    bids = bids_period.copy()
    
    width = bids["Capacity_MW"]
    height = bids[f"PRICE{hour}"]
    x_vals = find_x_pos(width)
    colors_mapped = bids['Group'].map(ENERGY_COLORS_DICT).to_list()
    # Temporary fix: convert the bids dataframe to a datasceince Table
    bids = Table.from_df(bids)
    bids = bids.with_column('Color', colors_mapped)
    group_colors = bids.group("Group", lambda x: x).select("Group", "Color")
    group_colors["Color"] = group_colors.apply(lambda x: x[0], "Color")
    
    #prepare the Marginal Cost to be a dashed line
    num_plants = len(width)
    height_mc = bids["Var_Cost_USDperMWH"]
    x_vec = np.zeros(num_plants * 2)
    h_vec = np.zeros(num_plants * 2)
    for i, (w, h) in enumerate(zip(width, height_mc)):
        h_vec[2*i] = h
        h_vec[2*i+1] = h
        if i == 0:
            x_vec[1] = w
        else: 
            x_vec[2*i] = x_vec[2*i - 1]
            x_vec[2*i + 1] = x_vec[2*i] + w
            
    # Make the plot
    plt.figure(figsize=(9,6))
    plt.bar(x_vals, height, width=width, color=bids['Color'], edgecolor = "black")
    plt.bar(x_vals, height_mc, width=width, color=bids['Color'], edgecolor = "black", alpha=.2)
    line_mc = plt.plot(x_vec, h_vec, '--k', label='Marginal Cost', linewidth=2)
    patches = []
    for row in group_colors.rows:
        patches += [mpatches.Patch(color=row.item("Color"), label=row.item("Group"))]    
    patches += line_mc
    plt.legend(handles=patches, bbox_to_anchor=(1.1,1))
    plt.title('Energy Market')
    plt.xlabel('Capacity_MW')
    plt.ylabel('Price')
    #plt.ylim((0,200))
    price_line_plot(price)
    demand_plot(demand)
    plt.show()

In [90]:
def portfolio_plot(bids, hour, period, demand, price, my_portfolio):    
    sorted_joined_table = Table.from_df(bids.copy())
    
    your_source = sorted_joined_table.where("Group", my_portfolio)
    width_yours = your_source.column("Capacity_MW")
    height_yours = your_source.column(f"PRICE{hour}")
    new_x_yours = find_x_pos(width_yours)
    label_yours = your_source.column("PLANT")
    colors_mapped = list(pd.Series(sorted_joined_table['Group']).map(ENERGY_COLORS_DICT))
    sorted_joined_table = sorted_joined_table.with_column('Color', colors_mapped)
    group_colors = sorted_joined_table.group("Group", lambda x: x).select("Group", "Color")
    group_colors["Color"] = group_colors.apply(lambda x: x[0], "Color")
    
    #prepare the Marginal Cost to be a dashed line
    num_plants = len(width_yours)
    height_mc = your_source.column("Var_Cost_USDperMWH")
    x_vec = np.zeros(num_plants * 2)
    h_vec = np.zeros(num_plants * 2)
    for i, (w, h) in enumerate(zip(width_yours, height_mc)):
        h_vec[2*i] = h
        h_vec[2*i+1] = h
        if i == 0:
            x_vec[1] = w
        else: 
            x_vec[2*i] = x_vec[2*i - 1]
            x_vec[2*i + 1] = x_vec[2*i] + w
            
    # Make the plot
    plt.figure(figsize=(11,6))
    plt.bar(new_x_yours, height_yours, width=width_yours, 
            color = ENERGY_COLORS_DICT[my_portfolio], edgecolor = "black")
    plt.bar(new_x_yours, height_mc, width=width_yours, 
            color = ENERGY_COLORS_DICT[my_portfolio], edgecolor = "black", alpha=.2)
    line_mc = plt.plot(x_vec, h_vec, '--k', label='Marginal Cost', linewidth=2)
    plt.title("Bids: " + my_portfolio)
    plt.xlabel('Capacity_MW')
    plt.ylabel('Price')
    for new_x_i, height_i, label_i in zip(new_x_yours, height_yours, label_yours):
        plt.text(new_x_i, height_i, label_i, ha='center', va='bottom', fontsize=8)
    price_line_plot(price)
            
    #the marginal plants should indicate how much capacity they produce
    nonmarg_capacity = sum(sorted_joined_table.where(f"PRICE{hour}", are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where(f"PRICE{hour}", are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    curr_capacity = 0
    
    for i, (w, h) in enumerate(zip(width_yours, height_yours)):
        if h == price:
            x_val = curr_capacity + (w * marg_proportion)
            x_vec = [x_val, x_val]
            h_vec = [0, h]
            plt.plot(x_vec, h_vec, '--k', linewidth=1)
        elif h > price:
            break
        curr_capacity += w
    
    plt.axvline(x=curr_capacity, color='k', linewidth = 2)
    
    plt.show()

## 5.2 Putting it all together
Demand, Bids, Market Price, Profits, and Emissions

Here is the main wrapper function -- this function takes all the inputs and settings and displays the market results:
- demand for this hour
- market clearing price
- portfolio profit
- portfolio emissions
- the market supply curve plot
- the portfolio-specific supply curve plot
- plant-specific info for the selected portfolio

In [91]:
def all_output(
    section=SECTIONS[0],
    hour=1,
    season='Fall',
    my_portfolio=portfolios[0],
    demand_sp=0,  
    carbon_p4=0,  #TODO just one carbon price
    carbon_p5=0,
    carbon_p6=0,
    update_my_bids="MC",  #TODO add bid_update_type = 'MC' or 'user_defined'
    update_others_bids="MC",
    alternative_bids=None,
    show_portfolio_specific=True,
    active_units=None,
):
    
    #TODO UPDATE THESE IN ALL FUNCTIONS
    period=2  # change period to season
    #TODO Remove my_portfolio and update user_defined_bids to take all bid updates 
    # or create an all_output function for market wide outputs
    print()

    # print that the current period is a pay-as-bid auction if it is
    if period in PAB_PERIODS:
        print("\033[1mNote:\033[0;0m The current period is a pay-as-bid auction.")
        print("")

    # print demand
    demand = demand_calc(hour, period, demand_sp)
    print("Demand: " + str(round(demand, 2)))

    # print price
    bids = get_bids(section)
    carbon_price_vec = [0, 0, 0, carbon_p4, carbon_p5, carbon_p6]
    carbon_price = carbon_price_vec[period - 1]
    bids["Original MC"] = bids["Var_Cost_USDperMWH"]
    #TODO is this double applying the carbon price?
    bids = adjust_by_cp(bids, hour, period, carbon_price)
    bids = user_defined_bids(
        bids,
        hour,
        period,
        my_portfolio,
        update_my_bids,
        update_others_bids,
        alternative_bids,
    )
    # filter out inactive units from capacity expansion section
    if active_units is not None:
        bids = bids[bids["UNIT NAME"].isin(active_units)]
    # Calculate the market clearing price
    bids_period = bids[bids["PERIOD"] == period].sort_values(f"PRICE{hour}", ascending=True)
    price = price_calc(bids_period, demand, hour, period)
    print("Price: $" + str(round(price, 2)))

    # print profits and emissions for this portfolio
    if show_portfolio_specific:
        my_profit = profit_calc(bids_period, hour, period, demand, price, my_portfolio)
        print(my_portfolio + " Profit: $" + str(round(my_profit, 2)))
        my_emissions = emissions_calc(bids_period, hour, demand, price, my_portfolio)
        print(my_portfolio + " Emissions: " + str(round(my_emissions, 2)) + " Tons CO2")
    
    # print market-wide profits and emissions
    else:
        total_emissions = marketwide_emissions_hour(bids_period, hour, price, demand)
        print("Market-wide Emissions:", total_emissions)
        total_profits = marketwide_profits_hour(bids, period, hour, price, demand)
        print("Market-wide Operating Profits:", total_profits)
        #TODO add market-wide capital investments in new plants
        print("Market-wide Capital Investments: PLACEHOLDER FOR NOW")
        
    # Display the market supply curve and clearing price
    market_plot(bids_period, hour, period, demand, price)

    # print portfolio specific information
    if show_portfolio_specific:
        portfolio_plot(bids_period, hour, period, demand, price, my_portfolio)

        # the marginal plants should indicate how much capacity they produce
        nonmarg_capacity = bids_period.loc[bids_period[f"PRICE{hour}"] < price, "Capacity_MW"].sum()
        marg_capacity = bids_period.loc[bids_period[f"PRICE{hour}"] == price, "Capacity_MW"].sum()
        marg_demand = demand - nonmarg_capacity
        marg_proportion = marg_demand / marg_capacity

        # display information about plants
        display_bids = bids_period[bids_period["Group"] == my_portfolio].copy()
        display_bids.rename(
            columns={
                "PLANT": "Plant",
                "Var_Cost_USDperMWH": "Adjusted MC",
                "Carbon_tonsperMWH": "Emissions Rate",
                f"PRICE{hour}": "Bid",
                "Capacity_MW": "Capacity",
                "FixedCst_OandM_perDay": "Fixed Cost per round",
            },
            inplace=True,
        )
        display_bids["Output"] = np.where(
            display_bids["Bid"] < price,
            display_bids["Capacity"],
            np.where(display_bids["Bid"] == price, display_bids["Capacity"] * marg_proportion, 0),
        ).round(1)
        display_bids.set_index(keys="Plant", inplace=True)
        display_bids.index.name = None
        print(f"\n\n {my_portfolio} Costs, Emissions Rates, Bids")
        display(
            display_bids[
                [
                    "Fixed Cost per round",
                    "Original MC",
                    "Emissions Rate",
                    "Adjusted MC",
                    "Bid",
                    "Capacity",
                    "Output",
                ]
            ]
        )

## 5.3 Define the alternative bids for all portfolios

### 5.3.1 Alternative Bids Helper Functions
To help setup a dictionary that can be modified by users, we need to print a base dictionary of all marginal costs.
Print a dictionary of portfolios and their plants/marginal costs -- these can be copied and pasted into the alternative_bids dictionary in the section 5.3.2.

In [92]:
# Print dictionary elements for marginal costs of all plants, by portfolio
print('ALTERNATIVE_BIDS = {')
for group in ESG.Group.unique():
    plants = ESG[ESG.Group == group]['UNIT NAME'].tolist()
    li = [f"'{plant}': {ESG.loc[(ESG['Group'] == group) & (ESG['UNIT NAME'] == plant), 'Var_Cost_USDperMWH'].values[0]}" for plant in plants]
    print(f"\t'{group}'" + ": {" + ", ".join(li) + "},")
print('}')

ALTERNATIVE_BIDS = {
	'Big Coal': {'FOUR CORNERS': 36.5, 'ALAMITOS 7': 73.72, 'HUNTINGTON BEACH 1&2': 40.5, 'HUNTINGTON BEACH 5': 66.5, 'REDONDO 5&6': 41.94, 'REDONDO 7&8': 41.94},
	'Big Gas': {'EL SEGUNDO 1&2': 44.83, 'EL SEGUNDO 3&4': 41.22, 'LONG BEACH': 52.5, 'NORTH ISLAND': 65.5, 'ENCINA': 41.67, 'KEARNY': 90.06, 'SOUTH BAY': 43.83},
	'Bay Views': {'MORRO BAY 1&2': 38.78, 'MORRO BAY 3&4': 36.61, 'MOSS LANDING 6': 32.56, 'MOSS LANDING 7': 32.56, 'OAKLAND': 61.17},
	'Beachfront': {'COOLWATER': 42.39, 'ETIWANDA 1-4': 42.67, 'ETIWANDA 5': 62.89, 'ELLWOOD': 75.61, 'MANDALAY 1&2': 39.06, 'MANDALAY 3': 52.06, 'ORMOND BEACH 1': 38.06, 'ORMOND BEACH 2': 38.06},
	'East Bay': {'PITTSBURGH 1-4': 40.94, 'PITTSBURGH 5&6': 36.61, 'PITTSBURGH 7': 59.72, 'CONTRA COSTA 4&5': 58.28, 'CONTRA COSTA 6&7': 39.5, 'POTRERO HILL': 69.83},
	'Old Timers': {'BIG CREEK': 0.0, 'MOHAVE 1': 34.5, 'MOHAVE 2': 34.5, 'HIGHGROVE': 49.61, 'SAN BERNADINO': 53.94},
	'Fossil Light': {'HUMBOLDT': 47.44, 'HELMS': 0.5, 'HUN

To help with merging, we need to convert the dictionary the user edits into a dataframe that can be merged with the bids data. This is used in the input to the `all_output` function, used below to plot the market bid curve.

In [93]:
def convert_dict_to_df(dict1):
    """Convert the alternative bids dictionary to a dataframe for merging, assumging all prices are the same for all hours."""
    # Create a dictionary to hold the columns of the dataframe
    dict2 = {'Group': [], 'UNIT NAME': [], 'PRICE1': [], 'PRICE2': [], 'PRICE3': [], 'PRICE4': []}
    # Iterate through the keys of the input dictionary and add the values to the columns of the new dictionary
    for k1 in dict1.keys():
        for k2 in dict1[k1].keys():
            dict2['Group'].append(k1)
            dict2['UNIT NAME'].append(k2)
            dict2['PRICE1'].append(dict1[k1][k2])
            dict2['PRICE2'].append(dict1[k1][k2])
            dict2['PRICE3'].append(dict1[k1][k2])
            dict2['PRICE4'].append(dict1[k1][k2])

    return pd.DataFrame(dict2)

### 5.3.2 Alternative Bids -- You can adjust these to test scenarios

Now we can define alternative bids to allow us to test difference scenarios -- if we change our bids, how does the bid curve and market clearing price change? What if other teams bid different from their marginal costs?

You can change the prices below to set the bids for your portfolio or other teams' portfolios.

These alternative bids are only used if `update_my_bids` or `update_others_bids` are True in the dropdown menus in the widgets in lower sections.

If you need to reset this section, just copy and paste from the ouptut in section 5.3.1.

In [94]:
ALTERNATIVE_BIDS = {
	'Big Coal': {'FOUR CORNERS': 36.5, 'ALAMITOS 7': 73.72, 'HUNTINGTON BEACH 1&2': 40.5, 'HUNTINGTON BEACH 5': 66.5, 'REDONDO 5&6': 41.94, 'REDONDO 7&8': 41.94},
	'Big Gas': {'EL SEGUNDO 1&2': 44.83, 'EL SEGUNDO 3&4': 41.22, 'LONG BEACH': 52.5, 'NORTH ISLAND': 65.5, 'ENCINA': 41.67, 'KEARNY': 90.06, 'SOUTH BAY': 43.83},
	'Bay Views': {'MORRO BAY 1&2': 38.78, 'MORRO BAY 3&4': 36.61, 'MOSS LANDING 6': 32.56, 'MOSS LANDING 7': 32.56, 'OAKLAND': 61.17},
	'Beachfront': {'COOLWATER': 42.39, 'ETIWANDA 1-4': 42.67, 'ETIWANDA 5': 62.89, 'ELLWOOD': 75.61, 'MANDALAY 1&2': 39.06, 'MANDALAY 3': 52.06, 'ORMOND BEACH 1': 38.06, 'ORMOND BEACH 2': 38.06},
	'East Bay': {'PITTSBURGH 1-4': 40.94, 'PITTSBURGH 5&6': 36.61, 'PITTSBURGH 7': 59.72, 'CONTRA COSTA 4&5': 58.28, 'CONTRA COSTA 6&7': 39.5, 'POTRERO HILL': 69.83},
	'Old Timers': {'BIG CREEK': 0.0, 'MOHAVE 1': 34.5, 'MOHAVE 2': 34.5, 'HIGHGROVE': 49.61, 'SAN BERNADINO': 53.94},
	'Fossil Light': {'HUMBOLDT': 47.44, 'HELMS': 0.5, 'HUNTERS POINT 1&2': 49.17, 'HUNTERS POINT 4': 75.89, 'DIABLO CANYON 1': 11.5},
}

## 5.4 Variable Definitions for the plotting widget below

`Section`: Section of the ESG game in which you are participating (defined by the table below).

| Code | Section Time  |
|---------|---------------|
| "W9" | Wednesday 9am|
| "T1" | Thursday 4pm - Aaron, Room 130|
| "T2" | Thursday 4pm - Max, Room 220|

`Hour`: Hour within the current round (ranges from 1 to 4).

`Period`: Round number (ranges from 0 to 6).

`my_portfolio`: Team portfolio of interest.

`demand_sp`: Adjustment to forecasted demand (or realized demand in past rounds). If value is between -1 and 1, gives a percentage change from forecasted demand. If value is greater than 1, gives a new value for demand in MWh.
For example, a value of 0.05 will assign demand to be (forecasted demand times 1.05). A value of 15000 will assign demand to be 15,000 MWh.
A value of -0.2 will assign demand to be 20% lower than forecasted demand.

`carbon_pX`: Assigns a carbon price in period X.
- Changing the carbon price will automatically adjust MC to include the per-unit tax burden, calculated using the carbon intensity of the plant.
- If changed for the current or future period, this will also automatically increase MC bid prices to inlcude the per-unit tax burden.

`update_my_bids`: 
If True, then allows you to alter the bids for the portfolio selected in `my_portfolio`. 
Alteration of bids can occur in the code in section 5.3.2. 
If MC, bids will revert to adjusted MC.
If False, bids will revert to adjusted MC, except for past rounds, where actual bids are used.

`update_others_bids`: 
If True, then allows you to alter the bids of the portfolios not selected in `my_portfolio`. 
Alteration of bids can occur in the code in section 5.3.2. 
If MC, bids will revert to adjusted MC.
If False, bids will revert to adjusted MC, except for past rounds, where actual bids are used.

## 5.5 Plot the Market and Portfolio

In [95]:
# Inject custom CSS into this cell to allow the full variable names to be displayed next to the dropdowns, otherwise they are cut off.
custom_css = """<style> .widget-label {min-width: 250px !important;} .widget-dropdown {width: 400px !important;} </style>"""
display(HTML(custom_css))

# Create an interface to allow the user to test adjustments to demand, carbon prices, and bids for different portfolios and hours
interact_obj = interact(lambda section, hour, period, my_portfolio, demand_sp, carbon_p4, carbon_p5, carbon_p6, update_my_bids, update_others_bids:
         all_output(section = section,
                    hour = hour,
                    period = period,
                    my_portfolio = my_portfolio,
                    demand_sp = demand_sp,
                    carbon_p4 = carbon_p4,
                    carbon_p5 = carbon_p5,
                    carbon_p6 = carbon_p6,
                    update_my_bids = update_my_bids,
                    update_others_bids = update_others_bids,
                    alternative_bids=convert_dict_to_df(ALTERNATIVE_BIDS)
         ),
         section = Dropdown(options=SECTIONS),
         hour = Dropdown(options=list(range(1,5))), 
         period = Dropdown(value = min(CURRENT_PERIOD, 6), options=list(range(0,7))),
         my_portfolio = Dropdown(options=np.unique(ESG["Group"])),
         demand_sp = BoundedFloatText(value=0, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width='350px')),
         carbon_p4 = BoundedFloatText(value=0, min = 0, max = 300, step=0.01, layout=widgets.Layout(width='350px')),
         carbon_p5 = BoundedFloatText(value=0, min = 0, max = 300, step=0.01, layout=widgets.Layout(width='350px')),
         carbon_p6 = BoundedFloatText(value=0, min = 0, max = 300, step=0.01, layout=widgets.Layout(width='350px')),
         update_my_bids = Dropdown(options=[False, True, 'MC'], description="Update my bids with alternative bids"),
         update_others_bids = Dropdown(options=[False, True, 'MC'], description="Update others' bids with alternative bids"))

HTML(value='<style> .widget-label {min-width: 250px !important;} .widget-dropdown {width: 400px !important;} <…

interactive(children=(Dropdown(description='section', options=('W9', 'T1', 'T2'), value='W9'), Dropdown(descri…

## 5.6 Total Round profits for one portfolio
Here's an example of how to write a function that would calculate the total round profits for a given portfolio.

In [96]:
def total_round_profit(section, period, my_portfolio, demand_sp, carbon_p4, carbon_p5, carbon_p6, update_my_bids, update_others_bids, alternative_bids):
    
    profits = []

    for hour in range(1, 5):
        # print demand
        demand = demand_calc(hour, period, demand_sp)
        
        # print price
        bids = get_bids(section)
        carbon_price_vec = [0, 0, 0, carbon_p4, carbon_p5, carbon_p6]
        carbon_price = carbon_price_vec[period - 1]
        bids['Original MC'] = bids['Var_Cost_USDperMWH']
        #TODO is this double applying the carbon price?
        bids = adjust_by_cp(bids, hour, period, carbon_price)
        bids = user_defined_bids(bids, hour, period, my_portfolio, update_my_bids, update_others_bids, alternative_bids)
        bids_period = bids[bids["PERIOD"] == period].sort_values(f"PRICE{hour}", ascending=True)
        price = price_calc(bids_period, demand, hour, period)
        
        # print profits and emissions
        my_profit = profit_calc(bids_period, hour, period, demand, price, my_portfolio)
        profits.append(my_profit)
    display(bids_period.sort_values(['PORTFOLIO', 'PLANT'], ascending=True))
    
    # Sum all hourly profits to get the total profit for the round
    total_profits = sum(profits)
    print(f'Total profit for {my_portfolio} in round {period}: ${total_profits:.2f}')
    return total_profits

In [97]:
total_round_profit(
    section='T1',
    period=6,
    my_portfolio='Beachfront',  # 'Bay Views', 'Beachfront', 'Big Coal', 'Big Gas', 'East Bay', 'Fossil Light', 'Old Timers'
    demand_sp=0,
    carbon_p4=88.24,
    carbon_p5=25,
    carbon_p6=24.04,
    update_my_bids=False,
    update_others_bids=False,
    alternative_bids=convert_dict_to_df(ALTERNATIVE_BIDS)
)
print()

,TEAM,TEAM_ID,PORTFOLIO,PORTFOLIO_ID,PLANT,PLANT_ID,PERIOD,PRICE1,PRICE2,PRICE3,PRICE4,Group,Group_num,UNIT NAME,Capacity_MW,Fuel_Cost_USDperMWH,Var_OandM_USDperMWH,Var_Cost_USDperMWH,Carbon_tonsperMWH,FixedCst_OandM_perDay,Plant_ID,Original MC
97,Debreu,4,Bay_Views,3,MORRO_BAY_1-2,31,6,38.78,38.78,38.78,49.5980,Bay Views,3,MORRO BAY 1&2,335,38.28,0.5,49.5980,0.45,2000,31,38.78
104,Debreu,4,Bay_Views,3,MORRO_BAY_3-4,32,6,36.61,36.61,36.61,46.9472,Bay Views,3,MORRO BAY 3&4,665,36.11,0.5,46.9472,0.43,4000,32,36.61
111,Debreu,4,Bay_Views,3,MOSS_LANDING_6,33,6,32.56,32.56,32.56,41.4548,Bay Views,3,MOSS LANDING 6,750,31.06,1.5,41.4548,0.37,8000,33,32.56
118,Debreu,4,Bay_Views,3,MOSS_LANDING_7,34,6,32.56,32.56,32.56,41.4548,Bay Views,3,MOSS LANDING 7,750,31.06,1.5,41.4548,0.37,8000,34,32.56
125,Debreu,4,Bay_Views,3,OAKLAND,35,6,61.17,61.17,61.17,78.4788,Bay Views,3,OAKLAND,150,60.67,0.5,78.4788,0.72,0,35,61.17
132,Arrow,1,Beachfront,4,COOLWATER,41,6,42.39,42.39,42.39,54.1696,Beachfront,4,COOLWATER,650,41.89,0.5,54.1696,0.49,2000,41,42.39
153,Arrow,1,Beachfront,4,ELLWOOD,44,6,75.61,75.61,75.61,97.0056,Beachfront,4,ELLWOOD,300,75.11,0.5,97.0056,0.89,0,44,75.61
139,Arrow,1,Beachfront,4,ETIWANDA_1-4,42,6,42.67,42.67,42.67,54.4496,Beachfront,4,ETIWANDA 1-4,850,41.17,1.5,54.4496,0.49,8000,42,42.67
146,Arrow,1,Beachfront,4,ETIWANDA_5,43,6,62.89,62.89,62.89,80.1988,Beachfront,4,ETIWANDA 5,150,61.39,1.5,80.1988,0.72,1000,43,62.89
160,Arrow,1,Beachfront,4,MANDALAY_1-2,45,6,39.06,39.06,39.06,49.6376,Beachfront,4,MANDALAY 1&2,300,37.56,1.5,49.6376,0.44,1000,45,39.06


Total profit for Beachfront in round 6: $16759.20



# 6 Forecasting Emissions under carbon prices

## 6.1 Emissions Forecasting Helper Functions

In [98]:
def predicted_emissions_456(section, D_R4_H1, D_R4_H2, D_R4_H3, D_R4_H4, D_R5_H1, D_R5_H2, D_R5_H3, D_R5_H4,
                            D_R6_H1, D_R6_H2, D_R6_H3, D_R6_H4, carbon_p4, carbon_p5, carbon_p6):
    demand_sp_vec = [D_R4_H1, D_R4_H2, D_R4_H3, D_R4_H4, D_R5_H1, D_R5_H2, D_R5_H3, D_R5_H4,
                  D_R6_H1, D_R6_H2, D_R6_H3, D_R6_H4]
    carbon_price_vec = [0, 0, 0, carbon_p4, carbon_p5, carbon_p6]
    emissions_dict = {'Round 4':{}, 'Round 5':{}, 'Round 6':{}, 'Total':{}}
    joined_table = get_bids(section)
    total_emissions = 0
    portfolio_emissions = 0
    for period_i in [4, 5, 6]:  
        total_emissions_period = 0
        carbon_price = carbon_price_vec[period_i - 1]

        for hour_i in [1, 2, 3, 4]:
            if (period_i < CURRENT_PERIOD) & (CURRENT_PERIOD < 8):
                joined_table_adj = user_defined_bids(joined_table, hour_i, period_i, 
                                                 my_portfolio = 'Big Coal', 
                                                 update_my_bids = False, update_others_bids = False,
                                                 alternative_bids = convert_dict_to_df(ALTERNATIVE_BIDS))
            else:
                joined_table_adj = adjust_by_cp(joined_table, hour_i, period_i, carbon_price)
            demand_sp = demand_sp_vec[4*(period_i - 4) + (hour_i - 1)]
            demand = demand_calc(hour_i, period_i, demand_sp)
            sorted_joined_table = joined_table_adj[joined_table_adj.PERIOD == period_i].sort_values(f"PRICE{hour_i}", ascending=True)
            price = price_calc(sorted_joined_table, demand, hour_i, period_i)

            for group in np.unique(ESG["Group"]):
                emissions_i = emissions_calc(sorted_joined_table, hour_i, demand, price, group)
                total_emissions_period += emissions_i
                if group not in emissions_dict['Round ' + str(period_i)].keys(): 
                    emissions_dict['Round ' + str(period_i)][group] = emissions_i
                else:
                    emissions_dict['Round ' + str(period_i)][group] += emissions_i
                if group not in emissions_dict['Total'].keys(): 
                    emissions_dict['Total'][group] = emissions_i
                else:
                    emissions_dict['Total'][group] += emissions_i
        
        emissions_dict[f'Round {period_i}']['Total Emissions'] = total_emissions_period
        total_emissions += total_emissions_period
        
    emissions_dict['Total']['Total Emissions'] = total_emissions
    
    output_df = pd.DataFrame(emissions_dict).round(2)
    
    return(output_df)

In [99]:
#function to get emissions across sections
def predicted_emissions_all_456(D_R4_H1, D_R4_H2, D_R4_H3, D_R4_H4, D_R5_H1, D_R5_H2, D_R5_H3, D_R5_H4,
                                D_R6_H1, D_R6_H2, D_R6_H3, D_R6_H4, carbon_p4, carbon_p5, carbon_p6):
    
    #need two dictionaries: one for the dataframe for all sections, one to store individual section dataframes
    emissions_dict_all = {'Round 4':{}, 'Round 5':{}, 'Round 6':{}, 'Total':{}}
    predicted_emissions_dfs = {}
    for section in SECTIONS:
        predicted_emissions_df = predicted_emissions_456(section = section,
                                                         D_R4_H1 = D_R4_H1,
                                                         D_R4_H2 = D_R4_H2,
                                                         D_R4_H3 = D_R4_H3,
                                                         D_R4_H4 = D_R4_H4,
                                                         D_R5_H1 = D_R5_H1,
                                                         D_R5_H2 = D_R5_H2,
                                                         D_R5_H3 = D_R5_H3,
                                                         D_R5_H4 = D_R5_H4,
                                                         D_R6_H1 = D_R6_H1,
                                                         D_R6_H2 = D_R6_H2,
                                                         D_R6_H3 = D_R6_H3,
                                                         D_R6_H4 = D_R6_H4,
                                                         carbon_p4 = carbon_p4,
                                                         carbon_p5 = carbon_p5,
                                                         carbon_p6 = carbon_p6)
        #add section dataframe to the dfs dictionary
        predicted_emissions_dfs[section] = predicted_emissions_df
        
        #get the emissions from each round by section for the total dataframe
        for period_i in range(4, 7): 
            emissions_dict_all['Round ' + str(period_i)][section] = predicted_emissions_df.loc['Total Emissions',
                                                                                               'Round ' + str(period_i)]
        emissions_dict_all['Total'][section] = predicted_emissions_df.loc['Total Emissions', 'Total']
        
    #create dataframe for all sections
    emissions_df = pd.DataFrame(emissions_dict_all).round(2)
    emissions_df.loc['Total Emissions',:] = emissions_df.sum(axis = 0)
    emissions_df.columns.name = 'All'
    display(emissions_df)
        
    #print individual section dataframes
    for section in SECTIONS:
        print(f'Predicted Emissions for Section {section}:')
        predicted_emissions_df = predicted_emissions_dfs[section]
        predicted_emissions_df.columns.name = section
        display(predicted_emissions_df)

## 6.2 Define your scenario
Finally, let's predict emissions under *competitive* bidding with the given carbon price.

`D_R4_H1` through `D_R6_H4`: demand for round 4, hour 1 through round 6, hour 4. These variables are used the same as `demand_sp` above, where 0 results no adjustment made to demand, a number between -1 and 1 results in a percentage change to demand, and a number larger than 1 results in demand being set at that value. For example, `D_R4_H1`=-0.3 results in decreasing the demand forecast for round 4, hour 1 by 30%, and `D_R4_H1`=17000 results in the demand forecast for round 4, hour 1 being replaced by 17,000 MWh. 

`carbon_p4` thorugh `carbon_p6`: the carbon price, in USD per tons of carbon, for each of the rounds 4 through 6. 0 means no carbon price, a negative number would be a subsidy on carbon emissions, and a positive number results in a tax on carbon.

Use the menu below to define a scenario by setting the demand and carbon prices for each round, and examine the resulting emissions.

In [100]:
# Print the total GHG budget for Rounds 4, 5, and 6
TOTAL_GHG_BUDGET = sum(govt_allocations_456)*3
print(f'Total GHG Budget for Rounds 4, 5, and 6: {TOTAL_GHG_BUDGET} Tons CO2  (as of 4/16/2024)')
print('Reminder: the GHG budget covers all three rounds and all three sections.')

text_box_width = '350px'

# Allow the user to set default demand adjustments for all hours in Rounds 4, 5, and 6
round4_demand_sp = 0
round5_demand_sp = 0
round6_demand_sp = 0
# Allow the user to set a default carbon price for both Rounds 5, and 6
round56_cp = 0

interact(lambda D_R4_H1, D_R4_H2, D_R4_H3, D_R4_H4, D_R5_H1, D_R5_H2, D_R5_H3, D_R5_H4, 
         D_R6_H1, D_R6_H2, D_R6_H3, D_R6_H4, carbon_p4, carbon_p5, carbon_p6:
         predicted_emissions_all_456(D_R4_H1 = D_R4_H1,
                                     D_R4_H2 = D_R4_H2,
                                     D_R4_H3 = D_R4_H3,
                                     D_R4_H4 = D_R4_H4,
                                     D_R5_H1 = D_R5_H1,
                                     D_R5_H2 = D_R5_H2,
                                     D_R5_H3 = D_R5_H3,
                                     D_R5_H4 = D_R5_H4,
                                     D_R6_H1 = D_R6_H1,
                                     D_R6_H2 = D_R6_H2,
                                     D_R6_H3 = D_R6_H3,
                                     D_R6_H4 = D_R6_H4,
                                     carbon_p4 = carbon_p4,
                                     carbon_p5 = carbon_p5,
                                     carbon_p6 = carbon_p6),
         D_R4_H1 = BoundedFloatText(value=round4_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R4_H2 = BoundedFloatText(value=round4_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R4_H3 = BoundedFloatText(value=round4_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R4_H4 = BoundedFloatText(value=round4_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R5_H1 = BoundedFloatText(value=round5_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R5_H2 = BoundedFloatText(value=round5_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R5_H3 = BoundedFloatText(value=round5_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R5_H4 = BoundedFloatText(value=round5_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R6_H1 = BoundedFloatText(value=round6_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R6_H2 = BoundedFloatText(value=round6_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R6_H3 = BoundedFloatText(value=round6_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         D_R6_H4 = BoundedFloatText(value=round6_demand_sp, min = -1, max = np.sum(ESG['Capacity_MW']), step=0.001, layout=widgets.Layout(width=text_box_width)),
         carbon_p4 = BoundedFloatText(value=0, min = 0, max = 300, step=0.01, layout=widgets.Layout(width=text_box_width)),
         carbon_p5 = BoundedFloatText(value=round56_cp, min = 0, max = 300, step=0.01, layout=widgets.Layout(width=text_box_width)),
         carbon_p6 = BoundedFloatText(value=round56_cp, min = 0, max = 300, step=0.01, layout=widgets.Layout(width=text_box_width)))
print('')

Total GHG Budget for Rounds 4, 5, and 6: 194568.0 Tons CO2  (as of 4/16/2024)
Reminder: the GHG budget covers all three rounds and all three sections.


interactive(children=(BoundedFloatText(value=0.0, description='D_R4_H1', layout=Layout(width='350px'), max=220…

In [101]:
print(f'Total GHG budget with offsets: {194568/0.95}')
print(f'Emissions over budget with offsets: {205811.21 - 194568/0.95}')
print(f'Emissions over budget without offsets: {205811.21 - 194568}')


Total GHG budget with offsets: 204808.4210526316
Emissions over budget with offsets: 1002.7889473683899
Emissions over budget without offsets: 11243.209999999992


Thanks to former GSIs who have contributed to this Notebook: Hal Gordon, Jesse Buchsbaum, Shelley He, Aaron Watt

Thanks for help from: Alec Kan, Alma Pineda, Aarish Irfan, Elaine Chien, Octavian Sima, and Eric Van Dusen.

# 7 Capacity Expansion
For this next activity, imagine you were hired by a statewide power planner who is trying to decide what the grid should look like over the next decade. The planner wants you to analyze the current power plant options and tell them which power plants to retire, which to keep, and which new power plants to bring online. The goal is to find the right mix of plants across the entire generator fleet that will meet demand at the least cost while also achieving our emissions goal.

## 7.1 Define the scenario
- All firms behave competitively (this is a simplification but a good place to start)
- Emissoins goal to reach: 18288 tons of CO2 annually.
- We have forecasts for the daily profile of electricity demand for each of the four seasons. 
  - We will assume these are perfect forecasts (i.e., that there is no concern about demand being less than or greater than what is forecasted).
  - We need to be able to meet demand at all hours in all four seasons.
- If a firm does not make a positive profit, they will not want to stay in the market. Therefore, we will assume that if any plant you have chosen to stay in the market makes negative profits, that plant will receive capacity payments to bring them up to zero profit so they will stay in the market. This capacity payment will be added to the overall costs of the sceneario you choose.


## 7.2 Define the options
- You get to decide to keep or retire any of the generating plants in the fleet.
- You get to decide to add any number of new Megawatts to the grid of several possible generator types.
- Each generator type has its own generating capacity, costs, and emissions intensity.

MAKE THIS A TABLE

**Generator Types**
- Coal
- Gas
- Nuclear
- Wind
- Solar

**Generator Characteristics**<br>
Each generator has the following characteristics
- Generating Capacity: 100 MW plants for all types
- Initial capitcal costs: the upfront fixed cost of building the plant
- Daily fixed costs: cost of operation and maintenance even if no energy is generated
- Variable Costs per MWh generated, which includes:
  - Fuel Costs per MWh
  - Variable Operating & Maintenance Costs per MWh
- Emissions Intensity, measured in tons of CO2 per MWh generated



## 7.3 Make a widget to simulate the market
NOTE: Current version only can exclude incumbent plants, still need to add new plants

In [102]:
header = widgets.HTML(value="<h1>ESG Capacity Expansion Simulation</h1>")
caption1 = widgets.HTML(value="<h2>Section 1: Choose the existing plants to keep or retire.</h2>")
caption2 = widgets.HTML(value="<h2>Section 2: Choose the number of new plants to bring online.</h2>")
placeholder_box = widgets.HTML(value="<h3>... PLACEHOLDER FOR ADDING NEW PLANTS...</h3>")
portfolios = sorted(portfolios)
plant_dict = {p: ESG[ESG["Group"] == p]["UNIT NAME"].sort_values().tolist() for p in portfolios}

# Create dropdown menus for season and hour
season_dropdown = widgets.Dropdown(
    options=SEASONS,
    value=SEASONS[2],
    description="Season:",
    disabled=False,
)
hour_dropdown = widgets.Dropdown(
    options=range(1, N_HOURS_PER_SEASON + 1),
    value=1,
    description="Hour:",
    disabled=False,
)

# Create checkboxes for reference and updating later
checkboxes = {
    group: [widgets.Checkbox(value=True, description=unit, indent=False)
            for unit in units]
    for group, units in plant_dict.items()
    }

# Create a grid of checkboxes for each portfolio
grids = [
    widgets.GridBox(checkboxes[g], layout=widgets.Layout(grid_template_columns="repeat(4, 200px)", indent=False))
    for g in portfolios
]

# Create an accordion section for each portfolio
accordion = widgets.Accordion(children=grids, titles=tuple(portfolios))

# Create an update button
run_button = widgets.Button(description="Run Market", button_style='danger')
output = widgets.Output()
update_counter = widgets.IntText(value=0)


# Display the widgets
display(header, season_dropdown, hour_dropdown, caption1, accordion, caption2, placeholder_box, run_button, output)


# Create a function that will update the display with units that are not checked
def on_button_clicked(b):
    # Clear the printed display
    output.clear_output()    
    excluded_units = []
    active_units = []
    with output:
        update_counter.value += 1
        print(f"\nUpdate {update_counter.value}")
        for portfolio in portfolios:
            if any([not checkbox.value for checkbox in checkboxes[portfolio]]):
                print(portfolio.upper())
                excluded = []
                for checkbox in checkboxes[portfolio]:
                    if checkbox.value:
                        active_units.append(checkbox.description)
                    else:
                        print(f"  --Excluding: {checkbox.description}")
                        excluded.append(checkbox.description)
                        excluded_units.append(checkbox.description)
                # Update accordion title
                accordion.set_title(portfolios.index(portfolio), f"{portfolio} - Excluded: {', '.join(excluded)}")
            else:
                active_units += plant_dict[portfolio]
                accordion.set_title(portfolios.index(portfolio), portfolio)
                
        # Run the market with the updated DataFrame
        all_output(
            # Section does not matter for this function
            section=SECTIONS[0],
            #TODO Add hour dropdown
            hour=hour_dropdown.value,
            #TODO Add season dropdown
            season=season_dropdown.value, #! "Fall"
            # Portfolio does not matter for this function
            my_portfolio=portfolios[0],
            # All demand and carbon price adjustments are set to 0
            demand_sp=0,
            carbon_p4=0,
            carbon_p5=0,
            carbon_p6=0,
            update_my_bids='MC',
            update_others_bids='MC',
            alternative_bids=convert_dict_to_df(ALTERNATIVE_BIDS),
            # Do not need portfolio specific information because we care about the entire market
            show_portfolio_specific=False,
            active_units=active_units
        )


# Assign the function to the button
run_button.on_click(on_button_clicked)


# Click the button
run_button.click()


#! ESG update worked, now just need to run market


HTML(value='<h1>ESG Capacity Expansion Simulation</h1>')

Dropdown(description='Season:', index=2, options=(1, 2, 3, 4), value=3)

Dropdown(description='Hour:', options=(1, 2, 3, 4), value=1)

HTML(value='<h2>Section 1: Choose the existing plants to keep or retire.</h2>')

Accordion(children=(GridBox(children=(Checkbox(value=True, description='MORRO BAY 1&2', indent=False), Checkbo…

HTML(value='<h2>Section 2: Choose the number of new plants to bring online.</h2>')

HTML(value='<h3>... PLACEHOLDER FOR ADDING NEW PLANTS...</h3>')

Button(button_style='danger', description='Run Market', style=ButtonStyle())

Output()

In [103]:
from typing import List

def on_click_plants(change:dict, buttons:List[widgets.ToggleButton]):
    active = change['new']
    state = 'Active' if active else 'Retired'
    for b in buttons:
        # b.description = f'{b.tooltip} ({state})'
        b.button_style = 'success' if active else ''
        b.icon = '' if active else 'times'
        b.value = active
    # print(change)
    
def on_click_portfolio(change:dict, button:widgets.ToggleButton, child_buttons:List[widgets.ToggleButton]):
    active = change['new']
    # button.description = f'{button.tooltip}'
    button.button_style = 'primary' if active else ''
    button.value = active
    on_click_plants(change, child_buttons)

def create_plant_button(unit:str, dummy=False, active=True, font_size='10px') -> widgets.ToggleButton:
    # Create toggle button
    b = widgets.ToggleButton(
        value=active,
        # description=f'{unit} (Active)',
        description=unit if dummy else unit.capitalize(),
        disabled=False,
        button_style='success' if active else '', # 'success', 'info', 'warning', 'danger' or ''
        tooltip=unit,
        icon='' if active else 'times',
        layout=widgets.Layout(height="auto", width="auto", white_space="pre"),
        style=dict(font_size=font_size)
    )
    # Tie the button click and toggle value to the on_click function
    # and allow it to update the button's attributes
    if not dummy:
        b.observe(lambda change: on_click_plants(change, [b]), names='value')
    return b

def create_portfolio_button(portfolio:str, plant_buttons:List[widgets.ToggleButton], dummy=False, active=True) -> widgets.ToggleButton:
    # Create toggle button
    b = widgets.ToggleButton(
        value=True,
        description=portfolio,
        disabled=False,
        button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
        tooltip=portfolio,
        layout=widgets.Layout(height="auto", width="auto", white_space="pre"),
        style=dict(font_size='large')
    )
    # Tie the button click and toggle value to the on_click function
    # and allow it to update the button's attributes
    if not dummy:
        b.observe(lambda change: on_click_portfolio(change, b, plant_buttons), names='value')
    return b

In [104]:
header = widgets.HTML(value="<h1>ESG Capacity Expansion Hourly Market Simulation</h1>")

portfolios = sorted(portfolios)
plant_dict = {p: ESG[ESG["Group"] == p]["UNIT NAME"].sort_values().tolist() for p in portfolios}

# Create dropdown menus for season and hour
season_dropdown = widgets.Dropdown(
    options=SEASONS,
    value=SEASONS[2],
    description="Season:",
    disabled=False,
)
hour_dropdown = widgets.Dropdown(
    options=range(1, N_HOURS_PER_SEASON + 1),
    value=1,
    description="Hour:",
    disabled=False,
)


#####################  PLANT RETIREMENT GRID  #######################
caption1 = widgets.HTML(
    value="<h2>Section 1: Choose the existing plants to keep or retire.</h2> "\
        "Click a plant to retire/activate it. "\
        "Click a portfolio to retire/activate all plants in that portfolio."
)
portfolios = sorted(portfolios)

n_portfolios = len(portfolios)
max_units = max(len(units) for units in plant_dict.values())
row_height = 40
n_rows = 2*n_portfolios
n_cols = 5
plant_font_size = '15px'

# Add a header row to display what the buttons represent
grid_header = GridspecLayout(1, n_cols, height=f"{row_height}px", width="1000px")
grid_header[0, 0] = create_portfolio_button("PORTFOLIO", [], dummy=True)
grid_header[0, 1] = create_plant_button("ACTIVE PLANT", dummy=True, font_size=plant_font_size)
grid_header[0, 2] = create_plant_button("RETIRED PLANT", dummy=True, active=False, font_size=plant_font_size)
# TODO make different colors for different types of plants (coal, gas, etc.)

n_rows = 2*n_portfolios
n_cols = 5
grid = GridspecLayout(n_portfolios*2, n_cols, height=f"{row_height*n_portfolios*2}px", width="1000px")
plant_button_lookup = {}
for i, portfolio in enumerate(portfolios):
    row = 2*i
    plant_button_lookup[portfolio] = []
    for j, plant in enumerate(plant_dict[portfolio]):
        col = j
        if j > 3:
            row = 2*i + 1
            col = j - 4
        plant_button = create_plant_button(plant, font_size=plant_font_size)
        plant_button_lookup[portfolio].append(plant_button)
        grid[row, col + 1] = plant_button
    grid[(2*i):(2*i+2), 0] = create_portfolio_button(portfolio, plant_button_lookup[portfolio])


#####################  NEW PLANT INVESTMENT  #######################
caption2 = widgets.HTML(value="<h2>Section 2: Choose the number of new plants to bring online.</h2>")
placeholder_box = widgets.HTML(value="<h3>... PLACEHOLDER FOR ADDING NEW PLANTS...</h3>")


#####################  RUN MARKET BUTTON  #######################

# Create an update button
run_button = widgets.Button(description="Run Market", button_style="danger")
output = widgets.Output()
update_counter = widgets.IntText(value=0)


# Create a function that will update the display with units that are not checked
def on_button_clicked(_):
    # Clear the printed display
    output.clear_output()
    retired_units = []
    active_units = []
    with output:
        update_counter.value += 1
        print(f"\nUpdate {update_counter.value}")
        for portfolio in portfolios:
            # If any units are retired, add them to the excluded list
            if any([not button.value for button in plant_button_lookup[portfolio]]):
                print(portfolio.upper())
                retired_temp = []
                for button in plant_button_lookup[portfolio]:
                    if button.value:
                        active_units.append(button.tooltip)
                    else:
                        print(f"  --Excluding: {button.description}")
                        retired_temp.append(button.tooltip)
                # Update accordion title
                retired_units += retired_temp

            # If all units are active, add all units to the active list
            else:
                active_units += plant_dict[portfolio]

        # Run the market with the updated DataFrame
        all_output(
            # Section does not matter for this function
            section=SECTIONS[0],
            # TODO Add hour dropdown
            hour=hour_dropdown.value,
            # TODO Add season dropdown
            season=season_dropdown.value,  #! "Fall"
            # Portfolio does not matter for this function
            my_portfolio=portfolios[0],
            # All demand and carbon price adjustments are set to 0
            demand_sp=0,
            carbon_p4=0,
            carbon_p5=0,
            carbon_p6=0,
            update_my_bids="MC",
            update_others_bids="MC",
            alternative_bids=convert_dict_to_df(ALTERNATIVE_BIDS),
            # Do not need portfolio specific information because we care about the entire market
            show_portfolio_specific=False,
            active_units=active_units,
        )


# Assign the function to the button
run_button.on_click(on_button_clicked)

# Click the button
run_button.click()


############################################


# Display the widgets
display(header, season_dropdown, hour_dropdown, caption1)
display(grid_header, grid)
display(caption2, placeholder_box)
display(run_button, output)

HTML(value='<h1>ESG Capacity Expansion Hourly Market Simulation</h1>')

Dropdown(description='Season:', index=2, options=(1, 2, 3, 4), value=3)

Dropdown(description='Hour:', options=(1, 2, 3, 4), value=1)

HTML(value='<h2>Section 1: Choose the existing plants to keep or retire.</h2> Click a plant to retire/activate…

GridspecLayout(children=(ToggleButton(value=True, button_style='primary', description='PORTFOLIO', layout=Layo…

GridspecLayout(children=(ToggleButton(value=True, button_style='success', description='Morro bay 1&2', layout=…

HTML(value='<h2>Section 2: Choose the number of new plants to bring online.</h2>')

HTML(value='<h3>... PLACEHOLDER FOR ADDING NEW PLANTS...</h3>')

Button(button_style='danger', description='Run Market', style=ButtonStyle())

Output()

In [105]:
#